In [ ]:
%pip install ortools

In [39]:
from ortools.sat.python import cp_model

def optimize_budget_with_weighted_loss(current_allocations, income, scale=1000):
    """
    Optimize budget allocation while penalizing deviation from current allocations.
    The penalty is weighted so that categories with lower original allocations incur a higher loss
    if changed. The effective objective is to minimize the weighted absolute deviation,
    which approximates minimizing relative deviation.

    Arguments:
      current_allocations: dict with keys corresponding to categories (e.g., 'savings', 'housing', etc.)
                           and current values.
      income: total monthly take-home income.
      scale: scaling factor for the weights (default 1000).

    Returns:
      A tuple (result, total_loss) where:
       - result is a dict with the optimized allocation for each category,
       - total_loss is the weighted loss.
    """
    model = cp_model.CpModel()
    
    # Define budget categories
    categories = ['transport', 'food', 'housing', 'insurance', 'investment', 'savings', 'total_needs', 'total_wants']
    variables = {}
    abs_devs = {}
    weights = {}
    for cat in categories:
        # Use current_allocations.get(cat, 0) to provide a default value if key is missing.
        curr = current_allocations.get(cat, 0)
        # For example, weight = round(scale / (curr + 1)).
        weights[cat] = scale / (curr + 1)
    
    # Suppose domain is in whole dollars.
    for cat in categories:
        # Decision variable for each category: allocation from 0 to income.
        variables[cat] = model.NewIntVar(0, income, cat)
        
        # Deviation variable (absolute difference) for each category.
        diff = model.NewIntVar(-income, income, f"diff_{cat}")
        model.Add(diff == variables[cat] - current_allocations.get(cat, 0))
        
        # Then obtain the absolute value of the difference.
        abs_devs[cat] = model.NewIntVar(0, income, f"abs_dev_{cat}")
        model.AddAbsEquality(abs_devs[cat], diff)
    
    # Total needs allocation constraint: sum of needs must be less than or equal to total_needs.
    model.Add(sum(variables[cat] for cat in ['transport', 'food', 'housing', 'insurance']) <= variables['total_needs'])

    # Total allocation constraint: sum of allocations must equal income.
    model.Add(sum(variables[cat] for cat in ['total_needs', 'total_wants', 'savings']) == income)
    
    # Add hard constraints as needed:
    model.Add(variables['savings'] >= int(0.2 * income))
    model.Add(variables['total_needs'] <= int(0.5 * income))
    model.Add(variables['total_wants'] <= int(0.3 * income))
    
    # Objective: minimize total quadratic deviation
    model.Minimize(sum(int(weights[cat] * 1000) * abs_devs[cat] for cat in categories))
    
    # Solve the model.
    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    
    if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        result = {cat: solver.Value(variables[cat]) for cat in categories}
        total_loss = sum(int(weights[cat] * 1000) * solver.Value(abs_devs[cat]) for cat in categories)
        return result, total_loss
    else:
        return None, None

# Example usage:
if __name__ == "__main__":
    # Assume current_allocations is what the user currently spends
    current_allocations = {
        'transport': 200,
        'food': 1000,
        'housing': 1200,
        'insurance': 50,
        'investment': 300,
        'savings': 1000,
        'total_needs': 2500,
        'total_wants': 500,  # other expenditures.
    }
    income = 4000
    solution, loss = optimize_budget_with_weighted_loss(current_allocations, income, scale=1000)
    if solution:
        print("Optimized Allocation:", solution)
        print("Total Weighted Loss:", loss)
    else:
        print("No feasible solution found.")


Optimized Allocation: {'transport': 200, 'food': 1000, 'housing': 750, 'insurance': 50, 'investment': 300, 'savings': 1500, 'total_needs': 2000, 'total_wants': 500}
Total Weighted Loss: 1073400


In [ ]:
from ortools.sat.python import cp_model

def create_huber_lookup_table(delta, max_dev):
    """
    Create a lookup table for the Huber loss function.
    
    For a deviation d, if d <= delta, the loss is 0.5 * d^2.
    Otherwise, the loss is delta * d - 0.5 * delta^2.
    
    We assume here that delta and deviations are integers.
    To avoid fractional values we compute:
    
        loss(d) = floor(0.5 * d^2)   for d <= delta
        loss(d) = delta*d - floor(0.5 * delta^2)  for d > delta
        
    You may also choose to scale these values if greater precision is required.
    """
    table = []
    delta2 = (delta * delta) // 2  # floor(0.5*delta^2)
    for d in range(max_dev + 1):
        if d <= delta:
            # Using integer division to approximate 0.5 * d^2.
            loss = (d * d) // 2
        else:
            loss = delta * d - delta2
        table.append(loss)
    return table

def optimize_budget_with_huber_loss(current_allocations, income, delta=50, max_dev=None):
    """
    Optimize budget allocation while penalizing deviations according to a Huber loss function.
    
    The Huber loss here is defined as:
      - (d^2)/2 for deviations d <= delta, and
      - delta*d - (delta^2)/2 for deviations larger than delta.
      
    We precompute a lookup table for d = 0,1,...,max_dev.
    
    Arguments:
      current_allocations: dict with keys corresponding to categories (e.g., 'savings', 'housing', etc.)
                           and current values.
      income: total monthly take-home income.
      delta: threshold for the Huber loss (integer) below which the loss is quadratic.
      max_dev: maximum deviation to consider in the lookup table; if None, defaults to income.
      
    Returns:
      A tuple (result, total_huber_loss), where result is a dict with optimized allocation
      for each category and total_huber_loss is the sum of Huber losses.
    """
    model = cp_model.CpModel()
    
    # Use income if max_dev is not provided.
    if max_dev is None:
        max_dev = income

    # Precompute the Huber loss lookup table.
    huber_table = create_huber_lookup_table(delta, max_dev)
    
    # Define budget categories.
    categories = ['transport', 'food', 'housing', 'insurance', 
                  'investment', 'savings', 'total_needs', 'total_wants']
    
    variables = {}
    huber_losses = {}
    
    # For each category, create:
    #  - a decision variable for the new allocation,
    #  - a difference variable for deviation,
    #  - an absolute deviation variable, and
    #  - a huber loss variable via the lookup table.
    for cat in categories:
        # Decision variable: allocation for each category (in whole dollars).
        variables[cat] = model.NewIntVar(0, income, cat)
        
        # Difference (can be negative) between new allocation and the current allocation.
        diff = model.NewIntVar(-income, income, f"diff_{cat}")
        model.Add(diff == variables[cat] - current_allocations.get(cat, 0))
        
        # Absolute deviation.
        abs_dev = model.NewIntVar(0, income, f"abs_dev_{cat}")
        model.AddAbsEquality(abs_dev, diff)
        
        # Restrict the deviation to our lookup table range.
        model.Add(abs_dev <= max_dev)
        
        # The huber loss value is selected from the lookup table.
        # Create an integer variable to hold the huber loss.
        huber_loss_var = model.NewIntVar(0, huber_table[-1], f"huber_{cat}")
        model.AddElement(abs_dev, huber_table, huber_loss_var)
        huber_losses[cat] = huber_loss_var

    # --- Budget constraints ---
    # Example: enforce that the sum of some individual needs is at most total_needs.
    model.Add(sum(variables[cat] for cat in ['transport', 'food', 'housing', 'insurance']) <= variables['total_needs'])
    
    # Total allocation constraint: total_needs, total_wants, and savings must add up to income.
    model.Add(sum(variables[cat] for cat in ['total_needs', 'total_wants', 'savings']) == income)
    
    # Some hard constraints (adjust based on your application):
    model.Add(variables['savings'] >= int(0.2 * income))
    model.Add(variables['total_needs'] <= int(0.5 * income))
    model.Add(variables['total_wants'] <= int(0.3 * income))
    
    # --- Objective ---
    # Minimize the total Huber loss over all categories.
    model.Minimize(sum(huber_losses[cat] for cat in categories))
    
    # Solve the model.
    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    
    if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        result = {cat: solver.Value(variables[cat]) for cat in categories}
        total_huber_loss = sum(solver.Value(huber_losses[cat]) for cat in categories)
        return result, total_huber_loss
    else:
        return None, None

# Example usage:
if __name__ == "__main__":
    # Example current allocations for each category.
    current_allocations = {
        'transport': 200,
        'food': 1000,
        'housing': 1200,
        'insurance': 50,
        'investment': 300,
        'savings': 1000,
        'total_needs': 2500,
        'total_wants': 500,
    }
    income = 4000
    # You can adjust delta as needed. Here, delta=50 means that deviations up to \$50 are penalized quadratically.
    solution, huber_loss = optimize_budget_with_huber_loss(current_allocations, income, delta=50)
    if solution:
        print("Optimized Allocation:", solution)
        print("Total Huber Loss:", huber_loss)
    else:
        print("No feasible solution found.")


In [ ]:
from ortools.sat.python import cp_model

def optimize_budget_with_weighted_quadratic_loss(current_allocations, income, scale=1000):
    """
    Optimize budget allocation while penalizing deviations from current allocations using a weighted quadratic loss.
    
    The loss for each category is:
      weight[cat] * (new_allocation[cat] - current_allocations[cat])^2
    where weight[cat] is computed as scale / (current_allocations[cat] + 1).
    
    Arguments:
      current_allocations: dict mapping category names (e.g., 'savings', 'housing', etc.) to their current values.
      income: total monthly take-home income.
      scale: a constant used to determine the weight for each category.
      
    Returns:
      A tuple (result, total_weighted_quad_loss) where:
        - result is a dict with the optimized allocation for each category,
        - total_weighted_quad_loss is the sum of the weighted quadratic losses.
    """
    model = cp_model.CpModel()
    
    # Define budget categories.
    categories = ['transport', 'food', 'housing', 'insurance',
                  'investment', 'savings', 'total_needs', 'total_wants']
    
    # Compute a weight for each category based on its current allocation.
    # A lower current allocation gives a higher weight.
    weights = {}
    for cat in categories:
        current_value = current_allocations.get(cat, 0)
        weights[cat] = scale / (current_value + 1)
        # Round to an integer (ensuring a minimum weight of 1).
        weights[cat] = max(1, int(round(weights[cat])))
    
    variables = {}
    quad_vars = {}  # quadratic loss variable for each category.
    weighted_quad_terms = []
    
    # Create decision variables and compute weighted quadratic loss for each category.
    for cat in categories:
        # Decision variable for each category: allocation from 0 to income.
        variables[cat] = model.NewIntVar(0, income, cat)
        
        # Deviation variable: difference between the new allocation and the current allocation.
        dev = model.NewIntVar(-income, income, f"dev_{cat}")
        current_val = current_allocations.get(cat, 0)
        model.Add(dev == variables[cat] - current_val)
        
        # Quadratic loss: compute the square of the deviation.
        quad = model.NewIntVar(0, income**2, f"quad_{cat}")
        model.AddMultiplicationEquality(quad, [dev, dev])
        quad_vars[cat] = quad
        
        # The weighted quadratic term is weight[cat] * quad.
        weighted_term = weights[cat] * quad
        weighted_quad_terms.append(weighted_term)
    
    # --- Budget Constraints ---
    # For example, enforce that the sum of transport, food, housing, and insurance
    # is at most the allocated total_needs.
    model.Add(sum(variables[cat] for cat in ['transport', 'food', 'housing', 'insurance'])
              <= variables['total_needs'])
    
    # Total allocation constraint: total_needs, total_wants, and savings must sum to income.
    model.Add(sum(variables[cat] for cat in ['total_needs', 'total_wants', 'savings']) == income)
    
    # Hard constraints (adjust these as needed):
    model.Add(variables['savings'] >= int(0.2 * income))
    model.Add(variables['total_needs'] <= int(0.5 * income))
    model.Add(variables['total_wants'] <= int(0.3 * income))
    
    # --- Objective: minimize the total weighted quadratic loss ---
    model.Minimize(sum(weighted_quad_terms))
    
    # Solve the model.
    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    
    if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        result = {cat: solver.Value(variables[cat]) for cat in categories}
        total_weighted_quad_loss = sum(solver.Value(quad_vars[cat]) * weights[cat] for cat in categories)
        return result, total_weighted_quad_loss
    else:
        return None, None

# Example usage:
if __name__ == "__main__":
    current_allocations = {
        'transport': 200,
        'food': 1000,
        'housing': 1200,
        'insurance': 50,
        'investment': 300,
        'savings': 1000,
        'total_needs': 2500,
        'total_wants': 500,
    }
    income = 4000
    solution, loss = optimize_budget_with_weighted_quadratic_loss(current_allocations, income, scale=1000)
    if solution:
        print("Optimized Allocation:")
        for k, v in solution.items():
            print(f"  {k}: {v}")
        print("Total Weighted Quadratic Loss:", loss)
    else:
        print("No feasible solution found.")


Optimized Allocation:
  transport: 160
  food: 800
  housing: 1000
  insurance: 40
  investment: 300
  savings: 1333
  total_needs: 2000
  total_wants: 667
Total Weighted Quadratic Loss: 506667


In [1]:
from ortools.sat.python import cp_model

def optimize_budget_with_weighted_quadratic_loss(current_allocations, income, scale=1000):
    """
    Optimize budget allocation while penalizing deviations from current allocations using a weighted quadratic loss.
    
    The loss for each category is:
      weight[cat] * (new_allocation[cat] - current_allocations[cat])^2
    where weight[cat] is computed as scale / (current_allocations[cat] + 1).
    
    Arguments:
      current_allocations: dict mapping category names (e.g., 'savings', 'housing', etc.) to their current values.
      income: total monthly take-home income.
      scale: a constant used to determine the weight for each category.
      
    Returns:
      A tuple (result, total_weighted_quad_loss) where:
        - result is a dict with the optimized allocation for each category,
        - total_weighted_quad_loss is the sum of the weighted quadratic losses.
    """
    model = cp_model.CpModel()
    
    # Define budget categories.
    categories = ['transport', 'food', 'housing', 'insurance', 'other_needs',
                  'investment', 'savings', 'total_needs', 'total_wants']
    
    # Compute a weight for each category based on its current allocation.
    # A lower current allocation gives a higher weight.
    weights = {}

    current_total_needs = 0
    for needs in ['transport', 'food', 'housing', 'insurance', 'other_needs']:
        current_total_needs += current_allocations.get(needs, 0)
    current_total_wants = income - current_total_needs - current_allocations.get('savings', 0)
    current_allocations['total_needs'] = current_total_needs
    current_allocations['total_wants'] = current_total_wants

    for cat in categories:
        current_value = current_allocations.get(cat, 0)
        weights[cat] = scale / (current_value + 1)
        # Round to an integer (ensuring a minimum weight of 1).
        weights[cat] = max(1, int(round(weights[cat])))
    
    variables = {}
    quad_vars = {}  # quadratic loss variable for each category.
    weighted_quad_terms = []
    
    # Create decision variables and compute weighted quadratic loss for each category.
    for cat in categories:
        # Decision variable for each category: allocation from 0 to income.
        variables[cat] = model.NewIntVar(0, income, cat)
        
        # Deviation variable: difference between the new allocation and the current allocation.
        dev = model.NewIntVar(-income, income, f"dev_{cat}")
        current_val = current_allocations.get(cat, 0)
        model.Add(dev == variables[cat] - current_val)
        
        # Quadratic loss: compute the square of the deviation.
        quad = model.NewIntVar(0, income**2, f"quad_{cat}")
        model.AddMultiplicationEquality(quad, [dev, dev])
        quad_vars[cat] = quad
        
        # The weighted quadratic term is weight[cat] * quad.
        weighted_term = weights[cat] * quad
        weighted_quad_terms.append(weighted_term)
    
    # --- Budget Constraints ---
    # For example, enforce that the sum of transport, food, housing, and insurance
    # is at most the allocated total_needs.
    model.Add(sum(variables[cat] for cat in ['transport', 'food', 'housing', 'insurance'])
              <= variables['total_needs'])
    
    # Total allocation constraint: total_needs, total_wants, and savings must sum to income.
    model.Add(sum(variables[cat] for cat in ['total_needs', 'total_wants', 'savings']) == income)
    
    # Hard constraints (adjust these as needed):
    model.Add(variables['savings'] >= int(0.2 * income))
    model.Add(variables['total_needs'] <= int(0.5 * income))
    model.Add(variables['total_wants'] <= int(0.3 * income))
    
    # --- Objective: minimize the total weighted quadratic loss ---
    model.Minimize(sum(weighted_quad_terms))
    
    # Solve the model.
    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    
    if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        result = {cat: solver.Value(variables[cat]) for cat in categories}
        total_weighted_quad_loss = sum(solver.Value(quad_vars[cat]) * weights[cat] for cat in categories)
        return result, total_weighted_quad_loss
    else:
        return None, None

# Example usage:
if __name__ == "__main__":
    current_allocations = {
        'transport': 200,
        'food': 1000,
        'housing': 1200,
        'insurance': 50,
        'other_needs': 50,
        'investment': 300,
        'savings': 1000,
    }
    income = 4000
    solution, loss = optimize_budget_with_weighted_quadratic_loss(current_allocations, income, scale=1000)
    if solution:
        print("Optimized Allocation:")
        for k, v in solution.items():
            print(f"  {k}: {v}")
        print("Total Weighted Quadratic Loss:", loss)
    else:
        print("No feasible solution found.")


load c:\repos\cs3263-project\.venv\Lib\site-packages\ortools\.libs\zlib1.dll...
load c:\repos\cs3263-project\.venv\Lib\site-packages\ortools\.libs\abseil_dll.dll...
load c:\repos\cs3263-project\.venv\Lib\site-packages\ortools\.libs\utf8_validity.dll...
load c:\repos\cs3263-project\.venv\Lib\site-packages\ortools\.libs\re2.dll...
load c:\repos\cs3263-project\.venv\Lib\site-packages\ortools\.libs\libprotobuf.dll...
load c:\repos\cs3263-project\.venv\Lib\site-packages\ortools\.libs\highs.dll...
load c:\repos\cs3263-project\.venv\Lib\site-packages\ortools\.libs\ortools.dll...
Optimized Allocation:
  transport: 160
  food: 800
  housing: 1000
  insurance: 40
  other_needs: 50
  investment: 300
  savings: 1333
  total_needs: 2000
  total_wants: 667
Total Weighted Quadratic Loss: 506667
